# 🎯 Emotion Classification Modeling Pipeline

This notebook implements a comprehensive modeling pipeline for emotion classification using the GoEmotions dataset. It replicates and extends the functionality from `emotion_xai/data/preprocessing.py` in an interactive environment.

## 📋 Notebook Overview

1. **Import Required Libraries** - Load necessary packages and modules
2. **Load and Explore Data** - Load raw data and perform initial analysis
3. **Data Preprocessing Pipeline** - Implement comprehensive data cleaning and transformation
4. **Feature Engineering** - Extract and prepare features for emotion analysis
5. **Model Training** - Train baseline and advanced models
6. **Model Evaluation** - Comprehensive performance analysis
7. **Save Processed Data** - Export preprocessed data and models to `data/processed/`

---

## 1. Import Required Libraries

Import all necessary libraries for data processing, modeling, and visualization.

In [1]:
# Core libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import re
import warnings
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any, Union
import time

# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, 
                           f1_score, classification_report, confusion_matrix)
from sklearn.model_selection import train_test_split
import joblib

# Text processing
import string
from collections import Counter

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Project modules - Add project root to path
import sys
sys.path.insert(0, '..')

# Import our custom modules
from emotion_xai.data.preprocessing import (
    load_dataset, assess_text_quality, clean_text, 
    filter_quality_issues, prepare_emotion_labels, 
    split_dataset, prepare_features, save_preprocessing_results,
    DataQualityMetrics, EMOTION_COLUMNS
)
from emotion_xai.models.baseline import BaselineModel, save_evaluation_results

# Configure display and warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('default')
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")
print(f"📊 Emotion columns available: {len(EMOTION_COLUMNS)}")

/Users/peter/AI_ML_Projects/emotion_xai_project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/peter/AI_ML_Projects/emotion_xai_project/.venv/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'Could not load this library: /Users/peter/AI_ML_Projects/emotion_xai_project/.venv/lib/python3.11/site-packages/torchvision/image.so'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/peter/AI_ML_Projects/emotion_xai_project/.venv/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python ext

✅ All libraries imported successfully!
📊 Emotion columns available: 28


## 2. Load and Explore Data

Load the GoEmotions dataset and perform initial exploration to understand the data structure.

In [2]:
# Setup paths
data_path = Path('../data/raw/goemotions.csv')
processed_data_dir = Path('../data/processed')
results_dir = Path('../results')

# Create directories if they don't exist
processed_data_dir.mkdir(parents=True, exist_ok=True)
(results_dir / 'metrics').mkdir(parents=True, exist_ok=True)
(results_dir / 'plots' / 'modeling').mkdir(parents=True, exist_ok=True)

# Load the dataset
print("📥 Loading GoEmotions dataset...")
start_time = time.time()
df = load_dataset(data_path)
load_time = time.time() - start_time

print(f"✅ Dataset loaded in {load_time:.2f} seconds")
print(f"📊 Dataset shape: {df.shape}")
print(f"📝 Columns: {list(df.columns[:10])}{'...' if len(df.columns) > 10 else ''}")

# Display basic information
print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)
print(f"Total samples: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print(f"Text column: {'text' in df.columns}")
print(f"Emotion columns: {len(EMOTION_COLUMNS)}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")

📥 Loading GoEmotions dataset...
📊 Dataset loaded: 211,225 samples with 28 emotion labels
✅ Dataset loaded in 0.59 seconds
📊 Dataset shape: (211225, 37)
📝 Columns: ['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear', 'admiration']...

DATASET OVERVIEW
Total samples: 211,225
Total columns: 37
Text column: True
Emotion columns: 28
Memory usage: 144.5 MB
📊 Dataset loaded: 211,225 samples with 28 emotion labels
✅ Dataset loaded in 0.59 seconds
📊 Dataset shape: (211225, 37)
📝 Columns: ['text', 'id', 'author', 'subreddit', 'link_id', 'parent_id', 'created_utc', 'rater_id', 'example_very_unclear', 'admiration']...

DATASET OVERVIEW
Total samples: 211,225
Total columns: 37
Text column: True
Emotion columns: 28
Memory usage: 144.5 MB


In [3]:
# Sample data preview
print("📋 Sample data preview:")
print("\nFirst 3 text samples:")
for i, text in enumerate(df['text'].head(3), 1):
    print(f"{i}. {text[:100]}{'...' if len(text) > 100 else ''}")

print(f"\n🏷️ Emotion columns present: {[col for col in EMOTION_COLUMNS if col in df.columns][:5]}...")
print(f"Total emotion columns: {len([col for col in EMOTION_COLUMNS if col in df.columns])}")

# Check for missing values
print(f"\n🔍 Missing values in text column: {df['text'].isna().sum()}")
print(f"Missing values in emotion columns: {df[EMOTION_COLUMNS].isna().sum().sum()}")

# Basic statistics
print(f"\n📊 Text length statistics:")
text_lengths = df['text'].astype(str).str.len()
print(f"Mean length: {text_lengths.mean():.1f} characters")
print(f"Median length: {text_lengths.median():.1f} characters")
print(f"Min length: {text_lengths.min()} characters")
print(f"Max length: {text_lengths.max()} characters")

# Emotion distribution preview
emotion_sums = df[EMOTION_COLUMNS].sum().sort_values(ascending=False)
print(f"\n🎭 Top 5 most frequent emotions:")
for emotion, count in emotion_sums.head().items():
    print(f"  {emotion}: {count:,} samples ({count/len(df)*100:.1f}%)")

📋 Sample data preview:

First 3 text samples:
1. That game hurt.
2.  >sexuality shouldn’t be a grouping category It makes you different from othet ppl so imo it fits th...
3. You do right, if you don't care then fuck 'em!

🏷️ Emotion columns present: ['admiration', 'amusement', 'anger', 'annoyance', 'approval']...
Total emotion columns: 28

🔍 Missing values in text column: 0
Missing values in emotion columns: 0

📊 Text length statistics:
Mean length: 69.3 characters
Median length: 67.0 characters
Min length: 2 characters
Max length: 703 characters

🎭 Top 5 most frequent emotions:
  neutral: 55,298 samples (26.2%)
  approval: 17,620 samples (8.3%)
  admiration: 17,131 samples (8.1%)
  annoyance: 13,618 samples (6.4%)
  gratitude: 11,625 samples (5.5%)


## 3. Data Preprocessing Pipeline

Implement the comprehensive preprocessing pipeline from `emotion_xai/data/preprocessing.py`.

In [4]:
# Step 1: Assess text quality
print("🔍 Step 1: Assessing text quality...")
quality_issues = assess_text_quality(df['text'])

print("📊 Quality Assessment Results:")
for issue_type, count in quality_issues.items():
    percentage = (count / len(df)) * 100
    print(f"  {issue_type}: {count:,} samples ({percentage:.2f}%)")

# Visualize quality issues
fig = px.bar(
    x=list(quality_issues.keys()),
    y=list(quality_issues.values()),
    title="Text Quality Issues Distribution",
    labels={'x': 'Issue Type', 'y': 'Number of Samples'},
    color=list(quality_issues.values()),
    color_continuous_scale='Reds'
)
fig.update_layout(showlegend=False, height=400)
fig.show()

# Calculate overall quality percentage
total_issues = sum(quality_issues.values())
clean_samples = len(df) - total_issues
quality_percentage = (clean_samples / len(df)) * 100
print(f"\n✨ Overall data quality: {quality_percentage:.2f}% clean samples")

🔍 Step 1: Assessing text quality...
📊 Quality Assessment Results:
  very_short: 57 samples (0.03%)
  very_long: 9 samples (0.00%)
  mostly_punctuation: 174 samples (0.08%)
  repeated_chars: 2,226 samples (1.05%)
  all_caps: 2,184 samples (1.03%)
  no_letters: 17 samples (0.01%)
  empty_null: 0 samples (0.00%)
📊 Quality Assessment Results:
  very_short: 57 samples (0.03%)
  very_long: 9 samples (0.00%)
  mostly_punctuation: 174 samples (0.08%)
  repeated_chars: 2,226 samples (1.05%)
  all_caps: 2,184 samples (1.03%)
  no_letters: 17 samples (0.01%)
  empty_null: 0 samples (0.00%)



✨ Overall data quality: 97.79% clean samples


In [5]:
# Step 2: Filter quality issues
print("🧹 Step 2: Filtering quality issues...")

# Apply quality filtering
filtered_df, quality_metrics = filter_quality_issues(df, remove_issues=True)

print(f"📊 Filtering results:")
print(f"  Original samples: {quality_metrics.total_samples:,}")
print(f"  Clean samples: {quality_metrics.clean_samples:,}")
print(f"  Removed samples: {quality_metrics.removed_samples:,}")
print(f"  Quality retention: {quality_metrics.quality_percentage:.2f}%")

# Compare before/after text lengths
original_lengths = df['text'].astype(str).str.len()
filtered_lengths = filtered_df['text'].astype(str).str.len()

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Original Data", "Filtered Data"),
    shared_yaxes=True
)

fig.add_trace(
    go.Histogram(x=original_lengths, name="Original", nbinsx=50, opacity=0.7),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=filtered_lengths, name="Filtered", nbinsx=50, opacity=0.7),
    row=1, col=2
)

fig.update_layout(
    title="Text Length Distribution: Before vs After Filtering",
    xaxis_title="Text Length (characters)",
    yaxis_title="Number of Samples",
    height=400
)
fig.show()

print(f"\n✅ Quality filtering completed. Proceeding with {len(filtered_df):,} samples.")

🧹 Step 2: Filtering quality issues...
🧹 Quality filtering: 211,225 → 211,008 samples
   Removed 217 (0.10%) problematic samples
📊 Filtering results:
  Original samples: 211,225
  Clean samples: 211,008
  Removed samples: 217
  Quality retention: 99.90%
🧹 Quality filtering: 211,225 → 211,008 samples
   Removed 217 (0.10%) problematic samples
📊 Filtering results:
  Original samples: 211,225
  Clean samples: 211,008
  Removed samples: 217
  Quality retention: 99.90%



✅ Quality filtering completed. Proceeding with 211,008 samples.


In [6]:
# Step 3: Split dataset
print("🔀 Step 3: Splitting dataset...")

# Split the data with stratification
splits = split_dataset(filtered_df, test_size=0.2, val_size=0.1, random_state=42)

train_df = splits['train']
val_df = splits['val']  
test_df = splits['test']

print(f"📊 Dataset splits:")
print(f"  Training: {len(train_df):,} samples ({len(train_df)/len(filtered_df)*100:.1f}%)")
print(f"  Validation: {len(val_df):,} samples ({len(val_df)/len(filtered_df)*100:.1f}%)")
print(f"  Test: {len(test_df):,} samples ({len(test_df)/len(filtered_df)*100:.1f}%)")

# Visualize split distribution
splits_data = {
    'Split': ['Train', 'Validation', 'Test'],
    'Samples': [len(train_df), len(val_df), len(test_df)],
    'Percentage': [len(train_df)/len(filtered_df)*100, 
                   len(val_df)/len(filtered_df)*100, 
                   len(test_df)/len(filtered_df)*100]
}

fig = px.pie(
    values=splits_data['Samples'],
    names=splits_data['Split'],
    title="Dataset Split Distribution"
)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

# Check emotion distribution across splits
print(f"\n🎭 Emotion distribution consistency across splits:")
train_emotions = train_df[EMOTION_COLUMNS].sum()
val_emotions = val_df[EMOTION_COLUMNS].sum()
test_emotions = test_df[EMOTION_COLUMNS].sum()

emotion_comparison = pd.DataFrame({
    'Train_Pct': train_emotions / len(train_df) * 100,
    'Val_Pct': val_emotions / len(val_df) * 100,
    'Test_Pct': test_emotions / len(test_df) * 100
})

print("Top 5 emotions distribution (%):")
for emotion in emotion_comparison.head().index:
    train_pct = emotion_comparison.loc[emotion, 'Train_Pct']
    val_pct = emotion_comparison.loc[emotion, 'Val_Pct']
    test_pct = emotion_comparison.loc[emotion, 'Test_Pct']
    print(f"  {emotion}: Train={train_pct:.1f}%, Val={val_pct:.1f}%, Test={test_pct:.1f}%")

🔀 Step 3: Splitting dataset...
📊 Dataset splits created:
   Train: 147,705 samples (70.0%)
   Val:   21,101 samples (10.0%)
   Test:  42,202 samples (20.0%)
📊 Dataset splits:
  Training: 147,705 samples (70.0%)
  Validation: 21,101 samples (10.0%)
  Test: 42,202 samples (20.0%)
📊 Dataset splits created:
   Train: 147,705 samples (70.0%)
   Val:   21,101 samples (10.0%)
   Test:  42,202 samples (20.0%)
📊 Dataset splits:
  Training: 147,705 samples (70.0%)
  Validation: 21,101 samples (10.0%)
  Test: 42,202 samples (20.0%)



🎭 Emotion distribution consistency across splits:
Top 5 emotions distribution (%):
  admiration: Train=8.1%, Val=8.2%, Test=8.1%
  amusement: Train=4.4%, Val=4.4%, Test=4.4%
  anger: Train=3.8%, Val=3.8%, Test=3.8%
  annoyance: Train=6.4%, Val=6.5%, Test=6.5%
  approval: Train=8.3%, Val=8.4%, Test=8.3%


## 4. Feature Engineering

Extract and prepare features for emotion classification using both conservative and aggressive text cleaning approaches.

In [7]:
# Step 4: Feature preparation
print("🔤 Step 4: Preparing text features...")

# Prepare features with conservative cleaning (default)
print("\n📝 Conservative text cleaning:")
train_texts_conservative = prepare_features(train_df, aggressive_cleaning=False)
val_texts_conservative = prepare_features(val_df, aggressive_cleaning=False)
test_texts_conservative = prepare_features(test_df, aggressive_cleaning=False)

# Prepare features with aggressive cleaning
print("\n🧹 Aggressive text cleaning:")
train_texts_aggressive = prepare_features(train_df, aggressive_cleaning=True)
val_texts_aggressive = prepare_features(val_df, aggressive_cleaning=True)
test_texts_aggressive = prepare_features(test_df, aggressive_cleaning=True)

# Compare cleaning approaches
print(f"\n🔍 Comparing cleaning approaches:")
print("Sample text transformations:")
sample_idx = 0
original_text = train_df.iloc[sample_idx]['text']
conservative_text = train_texts_conservative[sample_idx]
aggressive_text = train_texts_aggressive[sample_idx]

print(f"Original:     '{original_text}'")
print(f"Conservative: '{conservative_text}'")
print(f"Aggressive:   '{aggressive_text}'")

# Text length comparison
conservative_lengths = [len(text) for text in train_texts_conservative[:1000]]
aggressive_lengths = [len(text) for text in train_texts_aggressive[:1000]]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Conservative Cleaning", "Aggressive Cleaning")
)

fig.add_trace(
    go.Histogram(x=conservative_lengths, name="Conservative", nbinsx=30, opacity=0.7),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=aggressive_lengths, name="Aggressive", nbinsx=30, opacity=0.7),
    row=1, col=2
)

fig.update_layout(
    title="Text Length Distribution: Conservative vs Aggressive Cleaning",
    xaxis_title="Text Length (characters)",
    yaxis_title="Number of Samples",
    height=400,
    showlegend=False
)
fig.show()

print(f"\n✅ Feature preparation completed for both cleaning approaches")

🔤 Step 4: Preparing text features...

📝 Conservative text cleaning:
📝 Text features prepared: 147,705 samples
   Average length: 69.3 characters
   Aggressive cleaning: No
📝 Text features prepared: 21,101 samples
   Average length: 69.0 characters
   Aggressive cleaning: No
📝 Text features prepared: 147,705 samples
   Average length: 69.3 characters
   Aggressive cleaning: No
📝 Text features prepared: 21,101 samples
   Average length: 69.0 characters
   Aggressive cleaning: No
📝 Text features prepared: 42,202 samples
   Average length: 69.1 characters
   Aggressive cleaning: No

🧹 Aggressive text cleaning:
📝 Text features prepared: 42,202 samples
   Average length: 69.1 characters
   Aggressive cleaning: No

🧹 Aggressive text cleaning:
📝 Text features prepared: 147,705 samples
   Average length: 69.3 characters
   Aggressive cleaning: Yes
📝 Text features prepared: 147,705 samples
   Average length: 69.3 characters
   Aggressive cleaning: Yes
📝 Text features prepared: 21,101 samples
   


✅ Feature preparation completed for both cleaning approaches


In [8]:
# Step 5: Prepare emotion labels
print("🏷️ Step 5: Preparing emotion labels...")

# Prepare labels for all splits
train_labels = prepare_emotion_labels(train_df, EMOTION_COLUMNS)
val_labels = prepare_emotion_labels(val_df, EMOTION_COLUMNS)
test_labels = prepare_emotion_labels(test_df, EMOTION_COLUMNS)

print(f"📊 Label preparation results:")
print(f"  Training labels shape: {train_labels.shape}")
print(f"  Validation labels shape: {val_labels.shape}")
print(f"  Test labels shape: {test_labels.shape}")
print(f"  Number of emotions: {len(EMOTION_COLUMNS)}")

# Analyze label distribution
print(f"\n🎭 Multi-label analysis:")
train_labels_per_sample = train_labels.sum(axis=1)
val_labels_per_sample = val_labels.sum(axis=1)
test_labels_per_sample = test_labels.sum(axis=1)

print(f"  Average labels per sample:")
print(f"    Train: {train_labels_per_sample.mean():.2f}")
print(f"    Val: {val_labels_per_sample.mean():.2f}")
print(f"    Test: {test_labels_per_sample.mean():.2f}")

# Visualize label distribution
fig = make_subplots(
    rows=1, cols=3,
    subplot_titles=("Train", "Validation", "Test"),
    shared_yaxes=True
)

fig.add_trace(
    go.Histogram(x=train_labels_per_sample, name="Train", nbinsx=10, opacity=0.7),
    row=1, col=1
)
fig.add_trace(
    go.Histogram(x=val_labels_per_sample, name="Val", nbinsx=10, opacity=0.7),
    row=1, col=2
)
fig.add_trace(
    go.Histogram(x=test_labels_per_sample, name="Test", nbinsx=10, opacity=0.7),
    row=1, col=3
)

fig.update_layout(
    title="Distribution of Labels per Sample Across Splits",
    xaxis_title="Number of Labels",
    yaxis_title="Number of Samples",
    height=400,
    showlegend=False
)
fig.show()

# Show most frequent emotion combinations
multi_label_samples = train_labels_per_sample > 1
if multi_label_samples.sum() > 0:
    print(f"\n🔗 Multi-label samples: {multi_label_samples.sum():,} ({multi_label_samples.mean()*100:.1f}%)")
    
print(f"\n✅ Emotion labels prepared for all splits")

🏷️ Step 5: Preparing emotion labels...
🏷️  Emotion labels prepared: 28 emotions for 147,705 samples
   Multi-label samples: 25,078 (17.0%)
🏷️  Emotion labels prepared: 28 emotions for 21,101 samples
   Multi-label samples: 3,673 (17.4%)
🏷️  Emotion labels prepared: 28 emotions for 42,202 samples
   Multi-label samples: 7,226 (17.1%)
📊 Label preparation results:
  Training labels shape: (147705, 28)
  Validation labels shape: (21101, 28)
  Test labels shape: (42202, 28)
  Number of emotions: 28

🎭 Multi-label analysis:
  Average labels per sample:
    Train: 1.18
    Val: 1.19
    Test: 1.18



🔗 Multi-label samples: 25,078 (17.0%)

✅ Emotion labels prepared for all splits


## 5. Model Training

Train baseline models using both conservative and aggressive text cleaning approaches.

In [9]:
# Model 1: Conservative cleaning approach
print("🚀 Training Model 1: Conservative Text Cleaning")
print("="*50)

# Initialize model with optimized parameters for this dataset size
model_conservative = BaselineModel(
    max_features=5000,      # Balanced vocabulary size
    ngram_range=(1, 2),     # Unigrams and bigrams
    C=1.0,                  # Regularization strength
    max_iter=1000,          # Sufficient iterations
    random_state=42         # Reproducibility
)

# Train the conservative model
start_time = time.time()
model_conservative.fit(train_texts_conservative, train_labels, EMOTION_COLUMNS)
conservative_training_time = time.time() - start_time

print(f"✅ Conservative model training completed in {conservative_training_time:.1f} seconds")
print(f"📊 Model info: {model_conservative.training_info['n_features']} features, {model_conservative.training_info['vocabulary_size']} vocabulary")

🚀 Training Model 1: Conservative Text Cleaning
🚀 Training baseline model...
   📊 Training samples: 147705
   🏷️  Emotions: 28
   🔤 Vectorizing texts with TF-IDF...
   📈 TF-IDF shape: (147705, 5000)
   📝 Vocabulary size: 5000
   🎯 Training multi-label classifier...
   📈 TF-IDF shape: (147705, 5000)
   📝 Vocabulary size: 5000
   🎯 Training multi-label classifier...
   ✅ Training completed in 9.7 seconds
✅ Conservative model training completed in 9.7 seconds
📊 Model info: 5000 features, 5000 vocabulary
   ✅ Training completed in 9.7 seconds
✅ Conservative model training completed in 9.7 seconds
📊 Model info: 5000 features, 5000 vocabulary


In [10]:
# Model 2: Aggressive cleaning approach
print("\n🧹 Training Model 2: Aggressive Text Cleaning")
print("="*50)

# Initialize model with same parameters for fair comparison
model_aggressive = BaselineModel(
    max_features=5000,
    ngram_range=(1, 2),
    C=1.0,
    max_iter=1000,
    random_state=42
)

# Train the aggressive model
start_time = time.time()
model_aggressive.fit(train_texts_aggressive, train_labels, EMOTION_COLUMNS)
aggressive_training_time = time.time() - start_time

print(f"✅ Aggressive model training completed in {aggressive_training_time:.1f} seconds")
print(f"📊 Model info: {model_aggressive.training_info['n_features']} features, {model_aggressive.training_info['vocabulary_size']} vocabulary")

# Compare training times
print(f"\n⏱️ Training time comparison:")
print(f"  Conservative: {conservative_training_time:.1f} seconds")
print(f"  Aggressive: {aggressive_training_time:.1f} seconds")
print(f"  Difference: {abs(conservative_training_time - aggressive_training_time):.1f} seconds")

# Training summary
print(f"\n📈 Training Summary:")
print(f"  Dataset size: {len(train_df):,} samples")
print(f"  Features extracted: 5,000 TF-IDF features")
print(f"  Emotions to predict: {len(EMOTION_COLUMNS)}")
print(f"  Model type: One-vs-Rest Logistic Regression")
print(f"  Both models trained successfully! ✅")


🧹 Training Model 2: Aggressive Text Cleaning
🚀 Training baseline model...
   📊 Training samples: 147705
   🏷️  Emotions: 28
   🔤 Vectorizing texts with TF-IDF...
   📈 TF-IDF shape: (147705, 5000)
   📝 Vocabulary size: 5000
   🎯 Training multi-label classifier...
   📈 TF-IDF shape: (147705, 5000)
   📝 Vocabulary size: 5000
   🎯 Training multi-label classifier...
   ✅ Training completed in 4.2 seconds
✅ Aggressive model training completed in 4.2 seconds
📊 Model info: 5000 features, 5000 vocabulary

⏱️ Training time comparison:
  Conservative: 9.7 seconds
  Aggressive: 4.2 seconds
  Difference: 5.5 seconds

📈 Training Summary:
  Dataset size: 147,705 samples
  Features extracted: 5,000 TF-IDF features
  Emotions to predict: 28
  Model type: One-vs-Rest Logistic Regression
  Both models trained successfully! ✅
   ✅ Training completed in 4.2 seconds
✅ Aggressive model training completed in 4.2 seconds
📊 Model info: 5000 features, 5000 vocabulary

⏱️ Training time comparison:
  Conservative

## 6. Model Evaluation

Comprehensive evaluation of both models using multiple metrics and visualizations.

In [11]:
# Evaluate both models on validation set
print("📊 Evaluating models on validation set...")

# Conservative model evaluation
print("\n🔹 Conservative Model - Validation Results:")
conservative_val_metrics = model_conservative.evaluate(val_texts_conservative, val_labels)

# Aggressive model evaluation  
print("\n🔸 Aggressive Model - Validation Results:")
aggressive_val_metrics = model_aggressive.evaluate(val_texts_aggressive, val_labels)

# Compare validation performance
comparison_metrics = ['accuracy', 'f1_macro', 'f1_micro', 'precision_macro', 'recall_macro']

print(f"\n📈 Validation Performance Comparison:")
print(f"{'Metric':<20} {'Conservative':<12} {'Aggressive':<12} {'Difference':<12}")
print("-" * 60)

for metric in comparison_metrics:
    conservative_val = conservative_val_metrics[metric]
    aggressive_val = aggressive_val_metrics[metric]
    difference = aggressive_val - conservative_val
    print(f"{metric:<20} {conservative_val:<12.3f} {aggressive_val:<12.3f} {difference:<12.3f}")

# Visualize performance comparison
metrics_names = ['Accuracy', 'F1-Macro', 'F1-Micro', 'Precision', 'Recall']
conservative_values = [conservative_val_metrics[metric] for metric in comparison_metrics]
aggressive_values = [aggressive_val_metrics[metric] for metric in comparison_metrics]

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=metrics_names,
    y=conservative_values,
    mode='lines+markers',
    name='Conservative',
    line=dict(color='blue', width=3),
    marker=dict(size=8)
))
fig.add_trace(go.Scatter(
    x=metrics_names,
    y=aggressive_values,
    mode='lines+markers',
    name='Aggressive',
    line=dict(color='red', width=3),
    marker=dict(size=8)
))

fig.update_layout(
    title="Model Performance Comparison (Validation Set)",
    xaxis_title="Metrics",
    yaxis_title="Score",
    yaxis_range=[0, max(max(conservative_values), max(aggressive_values)) + 0.1],
    height=500
)
fig.show()

# Determine better model
if conservative_val_metrics['f1_macro'] > aggressive_val_metrics['f1_macro']:
    best_model = model_conservative
    best_texts_val = val_texts_conservative
    best_texts_test = test_texts_conservative
    best_approach = "Conservative"
else:
    best_model = model_aggressive
    best_texts_val = val_texts_aggressive
    best_texts_test = test_texts_aggressive
    best_approach = "Aggressive"

print(f"\n🏆 Best performing approach: {best_approach} (F1-macro: {max(conservative_val_metrics['f1_macro'], aggressive_val_metrics['f1_macro']):.3f})")

📊 Evaluating models on validation set...

🔹 Conservative Model - Validation Results:
📊 Evaluating model performance...
   🎯 Overall Accuracy: 0.126
   📊 F1-Score (macro): 0.161
   📊 F1-Score (micro): 0.221
   🏷️  Avg labels per sample: 1.19

🔸 Aggressive Model - Validation Results:
📊 Evaluating model performance...
   🎯 Overall Accuracy: 0.126
   📊 F1-Score (macro): 0.161
   📊 F1-Score (micro): 0.221
   🏷️  Avg labels per sample: 1.19

🔸 Aggressive Model - Validation Results:
📊 Evaluating model performance...
   🎯 Overall Accuracy: 0.125
   📊 F1-Score (macro): 0.161
   📊 F1-Score (micro): 0.221
   🏷️  Avg labels per sample: 1.19

📈 Validation Performance Comparison:
Metric               Conservative Aggressive   Difference  
------------------------------------------------------------
accuracy             0.126        0.125        -0.000      
f1_macro             0.161        0.161        -0.000      
f1_micro             0.221        0.221        -0.000      
precision_macro      0.6


🏆 Best performing approach: Conservative (F1-macro: 0.161)


In [12]:
# Final evaluation on test set
print(f"🎯 Final evaluation of {best_approach} model on test set...")

# Test set evaluation
test_metrics = best_model.evaluate(best_texts_test, test_labels)

print(f"\n📊 Test Set Performance ({best_approach} Model):")
print(f"  Accuracy: {test_metrics['accuracy']:.3f}")
print(f"  F1-Score (macro): {test_metrics['f1_macro']:.3f}")
print(f"  F1-Score (micro): {test_metrics['f1_micro']:.3f}")
print(f"  Precision (macro): {test_metrics['precision_macro']:.3f}")
print(f"  Recall (macro): {test_metrics['recall_macro']:.3f}")

# Check against PROJECT_PLAN requirements
target_f1 = 0.6
meets_f1_requirement = test_metrics['f1_macro'] > target_f1
meets_time_requirement = best_model.training_info['training_time_seconds'] < 600

print(f"\n🎯 PROJECT_PLAN Requirements Check:")
print(f"  F1-Score > 0.6: {'✅' if meets_f1_requirement else '❌'} (Actual: {test_metrics['f1_macro']:.3f})")
print(f"  Training < 10 min: {'✅' if meets_time_requirement else '❌'} (Actual: {best_model.training_info['training_time_seconds']:.1f}s)")

# Per-emotion performance analysis
print(f"\n🎭 Top performing emotions (F1-Score):")
emotion_f1_scores = [(emotion, metrics['f1']) for emotion, metrics in test_metrics['per_emotion'].items()]
emotion_f1_scores.sort(key=lambda x: x[1], reverse=True)

for i, (emotion, f1_score) in enumerate(emotion_f1_scores[:5], 1):
    print(f"  {i}. {emotion}: {f1_score:.3f}")

print(f"\n🎭 Challenging emotions (lowest F1-Score):")
for i, (emotion, f1_score) in enumerate(emotion_f1_scores[-5:], 1):
    print(f"  {i}. {emotion}: {f1_score:.3f}")

# Create per-emotion performance visualization
emotions = [item[0] for item in emotion_f1_scores]
f1_scores = [item[1] for item in emotion_f1_scores]

fig = px.bar(
    x=f1_scores,
    y=emotions,
    orientation='h',
    title=f"Per-Emotion F1-Scores ({best_approach} Model)",
    labels={'x': 'F1-Score', 'y': 'Emotions'},
    color=f1_scores,
    color_continuous_scale='RdYlGn'
)
fig.update_layout(height=800, yaxis={'categoryorder': 'total ascending'})
fig.show()

print(f"\n✅ Model evaluation completed!")

🎯 Final evaluation of Conservative model on test set...
📊 Evaluating model performance...
   🎯 Overall Accuracy: 0.127
   📊 F1-Score (macro): 0.156
   📊 F1-Score (micro): 0.217
   🏷️  Avg labels per sample: 1.18

📊 Test Set Performance (Conservative Model):
  Accuracy: 0.127
  F1-Score (macro): 0.156
  F1-Score (micro): 0.217
  Precision (macro): 0.520
  Recall (macro): 0.107

🎯 PROJECT_PLAN Requirements Check:
  F1-Score > 0.6: ❌ (Actual: 0.156)
  Training < 10 min: ✅ (Actual: 9.7s)

🎭 Top performing emotions (F1-Score):
  1. gratitude: 0.779
  2. love: 0.484
  3. amusement: 0.405
  4. admiration: 0.363
  5. optimism: 0.254

🎭 Challenging emotions (lowest F1-Score):
  1. disappointment: 0.021
  2. disapproval: 0.018
  3. realization: 0.014
  4. grief: 0.000
  5. relief: 0.000
   🎯 Overall Accuracy: 0.127
   📊 F1-Score (macro): 0.156
   📊 F1-Score (micro): 0.217
   🏷️  Avg labels per sample: 1.18

📊 Test Set Performance (Conservative Model):
  Accuracy: 0.127
  F1-Score (macro): 0.156



✅ Model evaluation completed!


## 7. Save Processed Data

Save all preprocessed data, trained models, and evaluation results to the `data/processed/` directory for future use.

In [16]:
# Save processed datasets
print("💾 Saving processed datasets to data/processed/...")

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Save train, validation, and test splits
train_df.to_csv(processed_data_dir / f'train_data_{timestamp}.csv', index=False)
val_df.to_csv(processed_data_dir / f'val_data_{timestamp}.csv', index=False)
test_df.to_csv(processed_data_dir / f'test_data_{timestamp}.csv', index=False)

print(f"✅ Dataset splits saved:")
print(f"  Train: {processed_data_dir}/train_data_{timestamp}.csv ({len(train_df):,} samples)")
print(f"  Val: {processed_data_dir}/val_data_{timestamp}.csv ({len(val_df):,} samples)")
print(f"  Test: {processed_data_dir}/test_data_{timestamp}.csv ({len(test_df):,} samples)")

# Save processed text features
processed_features = {
    'train_texts_conservative': train_texts_conservative,
    'val_texts_conservative': val_texts_conservative,
    'test_texts_conservative': test_texts_conservative,
    'train_texts_aggressive': train_texts_aggressive,
    'val_texts_aggressive': val_texts_aggressive,
    'test_texts_aggressive': test_texts_aggressive,
    'train_labels': train_labels.tolist(),
    'val_labels': val_labels.tolist(),
    'test_labels': test_labels.tolist(),
    'emotion_columns': EMOTION_COLUMNS,
    'preprocessing_info': {
        'timestamp': timestamp,
        'original_samples': len(df),
        'filtered_samples': len(filtered_df),
        'quality_retention': quality_metrics.quality_percentage,
        'train_size': len(train_df),
        'val_size': len(val_df),
        'test_size': len(test_df)
    }
}

import pickle
features_file = processed_data_dir / f'processed_features_{timestamp}.pkl'
with open(features_file, 'wb') as f:
    pickle.dump(processed_features, f)

print(f"✅ Processed features saved: {features_file}")

# Helper function to convert NumPy types to native Python types for JSON serialization
def convert_numpy_types(obj):
    """Convert NumPy data types to native Python types for JSON serialization."""
    import numpy as np
    if isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

# Save quality metrics
quality_metrics_dict = quality_metrics.to_dict()
quality_metrics_dict['timestamp'] = timestamp

# Convert NumPy types to native Python types
quality_metrics_dict = convert_numpy_types(quality_metrics_dict)

quality_file = processed_data_dir / f'quality_metrics_{timestamp}.json'
with open(quality_file, 'w') as f:
    json.dump(quality_metrics_dict, f, indent=2)

print(f"✅ Quality metrics saved: {quality_file}")

💾 Saving processed datasets to data/processed/...
✅ Dataset splits saved:
  Train: ../data/processed/train_data_20251128_045051.csv (147,705 samples)
  Val: ../data/processed/val_data_20251128_045051.csv (21,101 samples)
  Test: ../data/processed/test_data_20251128_045051.csv (42,202 samples)
✅ Processed features saved: ../data/processed/processed_features_20251128_045051.pkl
✅ Quality metrics saved: ../data/processed/quality_metrics_20251128_045051.json
✅ Dataset splits saved:
  Train: ../data/processed/train_data_20251128_045051.csv (147,705 samples)
  Val: ../data/processed/val_data_20251128_045051.csv (21,101 samples)
  Test: ../data/processed/test_data_20251128_045051.csv (42,202 samples)
✅ Processed features saved: ../data/processed/processed_features_20251128_045051.pkl
✅ Quality metrics saved: ../data/processed/quality_metrics_20251128_045051.json


In [17]:
# Save trained models
print(f"\n💾 Saving trained models to data/processed/...")

models_save_dir = processed_data_dir / 'models'
models_save_dir.mkdir(exist_ok=True)

# Save conservative model
conservative_model_file = models_save_dir / f'baseline_conservative_{timestamp}.joblib'
model_conservative.save(conservative_model_file)

# Save aggressive model  
aggressive_model_file = models_save_dir / f'baseline_aggressive_{timestamp}.joblib'
model_aggressive.save(aggressive_model_file)

print(f"✅ Models saved:")
print(f"  Conservative: {conservative_model_file}")
print(f"  Aggressive: {aggressive_model_file}")

# Save evaluation results
print(f"\n📊 Saving evaluation results...")

# Save to results directory (integrated with existing structure)
save_evaluation_results(
    metrics=test_metrics,
    model_info=best_model.training_info,
    results_dir=results_dir
)

# Also save comprehensive results to processed directory
comprehensive_results = {
    'timestamp': timestamp,
    'notebook_session': 'notebooks/02_modeling.ipynb',
    'best_approach': best_approach,
    'models': {
        'conservative': {
            'validation_metrics': conservative_val_metrics,
            'model_file': str(conservative_model_file),
            'training_time': conservative_training_time
        },
        'aggressive': {
            'validation_metrics': aggressive_val_metrics,
            'model_file': str(aggressive_model_file),
            'training_time': aggressive_training_time
        }
    },
    'final_test_metrics': test_metrics,
    'requirements_check': {
        'f1_target': target_f1,
        'f1_achieved': test_metrics['f1_macro'],
        'f1_requirement_met': meets_f1_requirement,
        'time_requirement_met': meets_time_requirement
    },
    'dataset_info': {
        'original_size': len(df),
        'processed_size': len(filtered_df),
        'train_size': len(train_df),
        'val_size': len(val_df),
        'test_size': len(test_df)
    }
}

results_file = processed_data_dir / f'modeling_results_{timestamp}.json'
with open(results_file, 'w') as f:
    json.dump(comprehensive_results, f, indent=2, default=str)

print(f"✅ Comprehensive results saved: {results_file}")

# Create summary file
summary_file = processed_data_dir / f'README_processed_{timestamp}.md'
with open(summary_file, 'w') as f:
    f.write(f"""# Processed Data Summary - {timestamp}

## Dataset Information
- Original samples: {len(df):,}
- Processed samples: {len(filtered_df):,}
- Quality retention: {quality_metrics.quality_percentage:.2f}%

## Data Splits
- Training: {len(train_df):,} samples
- Validation: {len(val_df):,} samples  
- Test: {len(test_df):,} samples

## Model Performance
- Best approach: {best_approach}
- Test F1-macro: {test_metrics['f1_macro']:.3f}
- Test accuracy: {test_metrics['accuracy']:.3f}

## Files Generated
- Dataset splits: train_data_{timestamp}.csv, val_data_{timestamp}.csv, test_data_{timestamp}.csv
- Features: processed_features_{timestamp}.pkl
- Models: baseline_conservative_{timestamp}.joblib, baseline_aggressive_{timestamp}.joblib
- Quality metrics: quality_metrics_{timestamp}.json
- Results: modeling_results_{timestamp}.json

Generated from: notebooks/02_modeling.ipynb
""")

print(f"✅ Summary documentation: {summary_file}")

print(f"\n🎉 All data successfully saved to data/processed/!")
print(f"📁 Total files created: 8 files")
print(f"💾 Ready for Phase 3: Transformer fine-tuning!")


💾 Saving trained models to data/processed/...
💾 Model saved to: ../data/processed/models/baseline_conservative_20251128_045051.joblib
💾 Model saved to: ../data/processed/models/baseline_aggressive_20251128_045051.joblib
✅ Models saved:
  Conservative: ../data/processed/models/baseline_conservative_20251128_045051.joblib
  Aggressive: ../data/processed/models/baseline_aggressive_20251128_045051.joblib

📊 Saving evaluation results...
📊 Evaluation results saved to: ../results/metrics/model_performance/baseline_evaluation_20251128_045911.json
✅ Comprehensive results saved: ../data/processed/modeling_results_20251128_045051.json
✅ Summary documentation: ../data/processed/README_processed_20251128_045051.md

🎉 All data successfully saved to data/processed/!
📁 Total files created: 8 files
💾 Ready for Phase 3: Transformer fine-tuning!


## 🎉 Modeling Pipeline Complete!

This notebook successfully implemented the complete emotion classification modeling pipeline:

### ✅ **Accomplishments**
- **Data Loading**: Loaded 211,225 GoEmotions samples
- **Quality Assessment**: Identified and filtered quality issues (99.9% retention)
- **Preprocessing**: Implemented both conservative and aggressive text cleaning
- **Feature Engineering**: Created TF-IDF features with optimized parameters
- **Model Training**: Trained baseline models with multi-label classification
- **Evaluation**: Comprehensive performance analysis and comparison
- **Data Persistence**: Saved all processed data to `data/processed/`

### 📊 **Key Results** 
- **Best Model**: {best_approach} text cleaning approach
- **Test Performance**: F1-macro = {test_metrics['f1_macro']:.3f}, Accuracy = {test_metrics['accuracy']:.3f}
- **Training Time**: {best_model.training_info['training_time_seconds']:.1f} seconds (meets < 10min requirement)
- **Data Quality**: 99.9% sample retention after quality filtering

### 📁 **Saved Outputs**
All processed data is available in `../data/processed/` for future use:
- Dataset splits (CSV files)
- Processed features (pickle file) 
- Trained models (joblib files)
- Evaluation results (JSON files)
- Documentation (README markdown)

### 🚀 **Next Steps**
Ready for **Phase 3: Transformer Fine-tuning** to improve F1-score from {test_metrics['f1_macro']:.3f} to target 0.6+

---
*Generated from: notebooks/02_modeling.ipynb*  
*Timestamp: {timestamp}*